In [ ]:
import geopandas as gpd
import ipywidgets as widgets
from ipyleaflet import Map, Popup, WMSLayer
from IPython.display import display
from ipywidgets import Layout
from pyproj import Transformer
from shapely.geometry import Point

GPKG_PATH = r"/home/jovyan/shared/common/teotil3/core_data/tidied/teotil3_data.gpkg"
POLY_PROPS = ['regine', 'regine_down', 'a_cat_land_km2']

# Initialize a map
m = Map(
    center=[60.4720, 8.4689], zoom=6, layout=Layout(width="800px", height="600px")
)  # coordinates for Norway

# Define WMS layer
wms = WMSLayer(
    url="https://nve.geodataonline.no/arcgis/services/Nedborfelt1/MapServer/WMSServer?",
    layers="REGINE_enhet",
    format="image/png",
    transparent=True,
    version="1.3.0",
    attribution="Norges vassdrags- og energidirektorat",
)

# Add WMS layer to the map
m.add_layer(wms)

# Create a transformer to convert coordinates
transformer = Transformer.from_crs("EPSG:4326", "EPSG:25833", always_xy=True)


# Function to handle click events
def handle_click(**kwargs):
    if kwargs.get("type") == "click":
        # Get the clicked coordinates and convert them to EPSG:25833
        lat, lon = kwargs.get("coordinates")
        x, y = transformer.transform(lon, lat)

        # Create a Point from the coordinates
        point = Point(x, y)

        # Read the GeoPackage file with the bounding box filter
        gdf = gpd.read_file(
            GPKG_PATH,
            layer="regines",
            driver="GPKG",
            bbox=[point.x, point.y, point.x, point.y],
        )

        # Check which polygon contains the point
        for _, row in gdf.iterrows():
            if row["geometry"].contains(point):
                # This polygon contains the point
                # Create a table with the properties of the polygon
                html = "<table>"
                for key, value in row.items():
                    if key in POLY_PROPS:
                        html += f"<tr><td>{key}</td><td>{value}</td></tr>"
                html += "</table>"

                # Create a popup with the table
                popup = Popup(
                    location=kwargs.get("coordinates"),
                    child=widgets.HTML(value=html),
                    close_button=False,
                    auto_close=False,
                    close_on_escape_key=False,
                )

                # Add the popup to the map
                m.add_layer(popup)


# Add click event handler to the map
m.on_interaction(handle_click)

# Display the map
m